Author: Jonathan Gong

In [14]:
import pandas as pd
import os
import sys
from sktime.annotation.hmm_learn import GMMHMM 
import numpy as np

# Data processing

In [7]:
csv_files = []
for file in os.listdir('../data/raw'):
    if file.endswith('.csv'):
        csv_files.append(pd.read_csv(f'../data/raw/{file}'))

C:\Users\jonat\AppData\Local\Temp\ipykernel_15640\895166927.py:4: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  csv_files.append(pd.read_csv(f'../data/raw/{file}'))


In [8]:
each_bgl =[]
for file in csv_files:
    each_df = file[["date", "bgl"]]
    each_df["date"] = pd.to_datetime(each_df["date"], format='mixed', utc=True)
    print(each_df)
    each_bgl.append(each_df.dropna(subset=["bgl"], how='all'))

C:\Users\jonat\AppData\Local\Temp\ipykernel_15640\1612716447.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  each_df["date"] = pd.to_datetime(each_df["date"], format='mixed', utc=True)


                           date    bgl
0     2024-07-01 05:02:39+00:00   98.0
1     2024-07-01 05:07:39+00:00  100.0
2     2024-07-01 05:12:39+00:00   98.0
3     2024-07-01 05:17:39+00:00   94.0
4     2024-07-01 05:22:40+00:00   94.0
...                         ...    ...
39622 2024-10-01 04:47:37+00:00   88.0
39623 2024-10-01 04:49:38+00:00   90.0
39624 2024-10-01 04:53:37+00:00   89.0
39625 2024-10-01 04:55:38+00:00   88.0
39626 2024-10-01 04:57:37+00:00   88.0

[39627 rows x 2 columns]
                           date    bgl
0     2024-07-01 04:02:32+00:00  115.0
1     2024-07-01 04:05:33+00:00  112.0
2     2024-07-01 04:08:33+00:00  116.0
3     2024-07-01 04:10:34+00:00  121.0
4     2024-07-01 04:13:36+00:00  122.0
...                         ...    ...
40101 2024-10-01 03:43:17+00:00  108.0
40102 2024-10-01 03:47:20+00:00  101.0
40103 2024-10-01 03:50:22+00:00   96.0
40104 2024-10-01 03:54:25+00:00  103.0
40105 2024-10-01 03:57:26+00:00  103.0

[40106 rows x 2 columns]


C:\Users\jonat\AppData\Local\Temp\ipykernel_15640\1612716447.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  each_df["date"] = pd.to_datetime(each_df["date"], format='mixed', utc=True)


In [9]:
each_bgl

[                           date    bgl
 0     2024-07-01 05:02:39+00:00   98.0
 1     2024-07-01 05:07:39+00:00  100.0
 2     2024-07-01 05:12:39+00:00   98.0
 3     2024-07-01 05:17:39+00:00   94.0
 4     2024-07-01 05:22:40+00:00   94.0
 ...                         ...    ...
 39622 2024-10-01 04:47:37+00:00   88.0
 39623 2024-10-01 04:49:38+00:00   90.0
 39624 2024-10-01 04:53:37+00:00   89.0
 39625 2024-10-01 04:55:38+00:00   88.0
 39626 2024-10-01 04:57:37+00:00   88.0
 
 [39518 rows x 2 columns],
                            date    bgl
 0     2024-07-01 04:02:32+00:00  115.0
 1     2024-07-01 04:05:33+00:00  112.0
 2     2024-07-01 04:08:33+00:00  116.0
 3     2024-07-01 04:10:34+00:00  121.0
 4     2024-07-01 04:13:36+00:00  122.0
 ...                         ...    ...
 40101 2024-10-01 03:43:17+00:00  108.0
 40102 2024-10-01 03:47:20+00:00  101.0
 40103 2024-10-01 03:50:22+00:00   96.0
 40104 2024-10-01 03:54:25+00:00  103.0
 40105 2024-10-01 03:57:26+00:00  103.0
 
 [39923 r

In [10]:
days_patients = []
for bgl in each_bgl:
    # (hour:minute:second)
    bgl['time'] = bgl['date'].dt.time
    # (year-month-day)
    bgl['day'] = bgl['date'].dt.date
    # unique days
    unique_days = bgl['day'].unique()
    day_data = []
    for i, day in enumerate(unique_days):
        bgl_day = bgl[bgl['day'] == day]
        day_data.append(bgl_day["bgl"])

    days_patients.append(day_data)

C:\Users\jonat\AppData\Local\Temp\ipykernel_15640\2013028078.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bgl['time'] = bgl['date'].dt.time
C:\Users\jonat\AppData\Local\Temp\ipykernel_15640\2013028078.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bgl['day'] = bgl['date'].dt.date


In [11]:
sequences = days_patients[0]
sequences = np.concatenate(sequences)

In [12]:
sequences


array([ 98., 100.,  98., ...,  89.,  88.,  88.])

In [15]:
model = GMMHMM(n_components=2, n_mix=2) 
model = model.fit(sequences) 

ModuleNotFoundError: No module named 'hmmlearn'